In [1]:
import sys
import os
import numpy as np
import pandas as pd

from configparser import ConfigParser
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMA

In [2]:
sys.path

['C:\\Users\\leada\\AlgoTradingStuff\\notebooks',
 'c:\\users\\leada\\anaconda3\\envs\\algotradingstuff\\python37.zip',
 'c:\\users\\leada\\anaconda3\\envs\\algotradingstuff\\DLLs',
 'c:\\users\\leada\\anaconda3\\envs\\algotradingstuff\\lib',
 'c:\\users\\leada\\anaconda3\\envs\\algotradingstuff',
 '',
 'c:\\users\\leada\\anaconda3\\envs\\algotradingstuff\\lib\\site-packages',
 'c:\\users\\leada\\anaconda3\\envs\\algotradingstuff\\lib\\site-packages\\win32',
 'c:\\users\\leada\\anaconda3\\envs\\algotradingstuff\\lib\\site-packages\\win32\\lib',
 'c:\\users\\leada\\anaconda3\\envs\\algotradingstuff\\lib\\site-packages\\Pythonwin',
 'c:\\users\\leada\\anaconda3\\envs\\algotradingstuff\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\leada\\.ipython']

In [3]:
if 'C:\\Users\\leada\\AlgoTradingStuff' not in sys.path:
    sys.path.append('C:\\Users\\leada\\AlgoTradingStuff')

In [4]:
from api_models.models import Account, get_account, get_accounts

In [5]:
ini = ConfigParser()
ini.read('../oanda.txt')
api_key = ini['oanda'].get('api_key')
base_url = ini['oanda'].get('base_url')
account_id = ini['oanda'].get('primary_account')

In [6]:
account = Account(api_key=api_key, base_url=base_url, account_id=account_id)

In [7]:
account

Account(api_key: <secret>, base_url: https://api-fxpractice.oanda.com/v3, account_id: 101-004-12979612-001

In [8]:
import datetime as dt
candles = account.get_candles('GBP_USD', 
                              since=dt.datetime(2020, 1, 3), 
                              to=dt.datetime(2020, 1, 12), 
                              granularity='M15',
                              count=1000)
candles
df = pd.DataFrame(columns=['time', 'open', 'high', 'low', 'close'])
for i, candle in enumerate(candles):
    t = candle.get('time')
    ohlc = candle.get('mid')
    o = np.float32(ohlc.get('o'))
    h = np.float32(ohlc.get('h'))
    l = np.float32(ohlc.get('l'))
    c = np.float32(ohlc.get('c'))
    df.loc[i, :] = [t, o, h, l, c]
df['time'] = pd.to_datetime(df['time'])
df.set_index('time', inplace=True)

In [9]:
df.head()

,open,high,low,close
time,,,,
2020-01-03 11:30:00+00:00,1.3075,1.3075,1.30537,1.30587
2020-01-03 11:45:00+00:00,1.30584,1.30618,1.3056,1.30591
2020-01-03 12:00:00+00:00,1.30588,1.30612,1.30544,1.30573
2020-01-03 12:15:00+00:00,1.3057,1.30671,1.3057,1.30646
2020-01-03 12:30:00+00:00,1.30644,1.30782,1.3064,1.30771


In [11]:
dfpct = df.pct_change().dropna()
dfpct.head()

,open,high,low,close
time,,,,
2020-01-03 11:45:00+00:00,-0.001270,-0.001010,0.000176,0.000031
2020-01-03 12:00:00+00:00,0.000031,-0.000046,-0.000123,-0.000138
2020-01-03 12:15:00+00:00,-0.000138,0.000452,0.000199,0.000559
2020-01-03 12:30:00+00:00,0.000567,0.000849,0.000536,0.000957
2020-01-03 12:45:00+00:00,0.000995,0.000321,0.000666,0.000252


In [12]:
adfuller(dfpct['open'])

(-33.519720249653034,
 0.0,
 0,
 998,
 {'1%': -3.4369193380671, '5%': -2.864440383452517, '10%': -2.56831430323573},
 -12209.164250924256)